In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import time
import pdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split

# Clustering
from sklearn.datasets import make_biclusters, make_checkerboard, make_classification
from sklearn.datasets import samples_generator as sg
from sklearn.cluster.bicluster import SpectralCoclustering, SpectralBiclustering
from sklearn.metrics import consensus_score

# Corr
from scipy.stats import spearmanr, pearsonr
from numpy import corrcoef

# Utils
file_path = os.getcwd()  # os.path.dirname(os.path.relpath(__file__))
utils_path = os.path.abspath(os.path.join(file_path, '..', '..', 'utils_py'))
sys.path.append(utils_path)
import utils_all as utils

# Feature importance
pfi_path = os.path.abspath(os.path.join(file_path, '..', 'pfi'))
sys.path.append(pfi_path)
import pfi
import pfi_utils

import warnings
warnings.filterwarnings('ignore')

SEED=0

Using TensorFlow backend.


In [2]:
OUTDIR = os.path.join(file_path, 'data')
print(OUTDIR)

/vol/ml/apartin/projects/feature-importance/apps/data


In [3]:
DATAPATH = '/vol/ml/apartin/Benchmarks/Data/Pilot1/'
FILENAME = 'matched_normal_samples.FPKM-UQ.csv'
nt_org = pd.read_csv(os.path.join(DATAPATH, FILENAME), sep=',')
print(nt_org.shape)

# data_val_org = pd.read_csv(os.path.join(datapath, 'P1B1.dev.test.csv'), sep=',')
# print(data_val_org.shape)

(1428, 60484)


In [16]:
NT_METAFILE = 'matched_normal_samples.metadata.tsv'
nt_meta = pd.read_csv(os.path.join(DATAPATH, NT_METAFILE), sep='\t', header=None)
print(nt_meta.shape)
nt_meta[:3]

(726, 7)


,0,1,2,3,4,5,6
0,aeec3005-25d7-45fc-a9ff-014d57960216,Prostate,Prostate Adenocarcinoma,Solid Tissue Normal,76a194e5-ba36-4e81-b0b4-6c22a69fe9b5.FPKM-UQ.txt,Primary Tumor,26b9d14a-da14-4062-8494-fb87b04e9d7b.FPKM-UQ.txt
1,c8fb94dd-c535-48f4-9419-aac2bf47bbb4,Uterus,Uterine Corpus Endometrial Carcinoma,Solid Tissue Normal,e8caa240-51e5-41c2-8340-32613692be11.FPKM-UQ.txt,Primary Tumor,788465ed-def3-4d72-ac49-6d2f0c19759e.FPKM-UQ.txt
2,de206681-3e27-4d4f-8669-aefb12b5b1e3,Head and Neck,Head and Neck Squamous Cell Carcinoma,Solid Tissue Normal,5e79c5a9-5341-44e1-95fb-392020416b0c.FPKM-UQ.txt,Primary Tumor,eaf69a94-f9b7-4177-b5eb-aa16cf102b96.FPKM-UQ.txt


In [17]:
nt_meta = nt_meta.rename(columns={1: 'origin', 2: 'ctype'})
# nt_meta = nt_meta.rename(columns={0: 'Sample', 1: 'origin', 2: 'ctype'})

In [18]:
nt_meta[:2]

,0,origin,ctype,3,4,5,6
0,aeec3005-25d7-45fc-a9ff-014d57960216,Prostate,Prostate Adenocarcinoma,Solid Tissue Normal,76a194e5-ba36-4e81-b0b4-6c22a69fe9b5.FPKM-UQ.txt,Primary Tumor,26b9d14a-da14-4062-8494-fb87b04e9d7b.FPKM-UQ.txt
1,c8fb94dd-c535-48f4-9419-aac2bf47bbb4,Uterus,Uterine Corpus Endometrial Carcinoma,Solid Tissue Normal,e8caa240-51e5-41c2-8340-32613692be11.FPKM-UQ.txt,Primary Tumor,788465ed-def3-4d72-ac49-6d2f0c19759e.FPKM-UQ.txt


In [19]:
nt_org[:2]

,Sample,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,...,ENSGR0000263980.4,ENSGR0000264510.4,ENSGR0000264819.4,ENSGR0000265658.4,ENSGR0000270726.4,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1
0,00511204-3512-4a5e-b664-60271e968903,406251.224803,298928.274199,727338.848986,68471.247940,16816.622522,80082.645290,438199.589403,629152.411235,524231.496729,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,011ee3e1-37bd-47c1-9092-a24dffd3d5f5,832125.341022,26835.365035,497535.852837,62538.020368,9615.885044,51523.282898,98399.389116,575054.212119,49998.117980,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
nt_meta[4][0]

'76a194e5-ba36-4e81-b0b4-6c22a69fe9b5.FPKM-UQ.txt'

In [15]:
any(nt_org['Sample'] == nt_meta[0][0])

False

In [46]:
set(nt_meta['Sample']).intersection(nt_org['Sample'])

set()

In [37]:
nt_org.merge(nt_meta, how='inner', on='Sample')

,Sample,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,...,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1,0,origin,ctype,3,4,5


In [ ]:
# Load data
print('Loading NT data ...')
nt = pd.read_csv(os.path.join(DATAPATH, FILENAME), sep=',')
mm = pd.read_csv(os.path.join(DATAPATH, GENEMAPFILE), sep='\t')

# Extract lincs cols from the whole dataset
nt = nt[['Sample'] + mm['gdc'].tolist()]

# Map lincs gene names and sort genes alphabetically
col_mapping = {mm.loc[g, 'gdc']: mm.loc[g, 'symbol'] for g in range(mm.shape[0])}
nt = nt.rename(columns=col_mapping)
nt = nt[['Sample'] + sorted(nt.columns[1:].tolist())]

# Shuffle and extract the target label
nt = nt.sample(n=nt.shape[0], axis=0, replace=False, random_state=SEED).reset_index(drop=True)
nt['Sample'] = nt['Sample'].map(lambda s: s.split('-')[-1]).values
nt.rename(columns={'Sample': 'y'}, inplace=True)

print(nt['y'].value_counts())
nt.to_csv(os.path.join(OUTDIR, f'{APP}_data'), sep='\t', index=False)

In [4]:
mm = pd.read_csv(os.path.join(datapath, 'lincs1000.tsv'), sep='\t')

In [5]:
data_train = data_train_org[['case_id', 'cancer_type'] + mm['gdc'].tolist()]  # Extract lincs from the whole dataset
data_val = data_val_org[['case_id', 'cancer_type'] + mm['gdc'].tolist()]  # Extract lincs from the whole dataset
print(data_train.shape)
print(data_val.shape)

(3000, 980)
(1000, 980)


In [6]:
# Map lincs gene names
col_mapping = {mm.loc[g, 'gdc']: mm.loc[g, 'symbol'] for g in range(mm.shape[0])}
data_train = data_train.rename(columns=col_mapping)
data_val = data_val.rename(columns=col_mapping)

In [7]:
data_train = data_train[['case_id', 'cancer_type'] + sorted(data_train.columns[2:].tolist())]
data_val = data_val[['case_id', 'cancer_type'] + sorted(data_val.columns[2:].tolist())]

In [8]:
data_train.drop(columns='case_id', inplace=True)
data_val.drop(columns='case_id', inplace=True)

In [9]:
data_train.rename(columns={'cancer_type': 'y'}, inplace=True)
data_val.rename(columns={'cancer_type': 'y'}, inplace=True)

In [10]:
data_train['y'].value_counts()

Lung Squamous Cell Carcinoma             300
Lung Adenocarcinoma                      300
Prostate Adenocarcinoma                  300
Brain Lower Grade Glioma                 300
Others                                   300
Uterine Corpus Endometrial Carcinoma     300
Thyroid Carcinoma                        300
Breast Invasive Carcinoma                300
Skin Cutaneous Melanoma                  300
Head and Neck Squamous Cell Carcinoma    300
Name: y, dtype: int64

In [11]:
data_val['y'].value_counts()

Uterine Corpus Endometrial Carcinoma     100
Lung Squamous Cell Carcinoma             100
Thyroid Carcinoma                        100
Breast Invasive Carcinoma                100
Lung Adenocarcinoma                      100
Skin Cutaneous Melanoma                  100
Head and Neck Squamous Cell Carcinoma    100
Prostate Adenocarcinoma                  100
Brain Lower Grade Glioma                 100
Others                                   100
Name: y, dtype: int64

In [21]:
y_train = data_train['y'].values
y_val = data_val['y'].values
y_enc = LabelEncoder()

In [22]:
y_train = y_enc.fit_transform(y_train)
y_val = y_enc.transform(y_val)

In [ ]:
y_encoded = pd.DataFrame(data={'label': np.arange(0, len(y_enc.classes_)), 'type': y_enc.classes_})
y_encoded.to_csv(os.path.join(OUTDIR, 'tc_y_enc_ff'), sep='\t', index=False)

In [37]:
data_train['y'] = y_train
data_val['y'] = y_val

In [42]:
data_train = data_train.sample(n=data_train.shape[1], replace=False, random_state=SEED).reset_index(drop=True)
data_val = data_val.sample(n=data_val.shape[1], replace=False, random_state=SEED).reset_index(drop=True)

In [46]:
# data_train.to_csv(os.path.join(file_path, 'data', 'P1B1.dev.train.lincs.csv'), index=False)
# data_val.to_csv(os.path.join(file_path, 'data', 'P1B1.dev.test.lincs.csv'), index=False)

data_train.to_csv(os.path.join(OUTDIR, 'P1B1.dev.train.lincs'), sep='\t', index=False)
data_val.to_csv(os.path.join(OUTDIR, 'P1B1.dev.test.lincs'), sep='\t', index=False)